##  감성체인 생성

In [16]:
import os
import time
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from concurrent.futures import ThreadPoolExecutor, as_completed
from langchain.chains import LLMChain  # LLMChain을 사용하여 Chain을 구성합니다
import logging

In [4]:
# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

In [5]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# OpenAI LLM 모델 설정
llm = ChatOpenAI(openai_api_key=api_key, model_name="gpt-4-turbo-preview", temperature=0.3)

# 감성 분석 기본 프롬프트
prompt_template = PromptTemplate.from_template(
    """你是一名中国社会科学家，你的任务是分析中国新闻报道，进行中国媒体对关于“美日韩”三边关系的报道文章的情感分析，**请在分析时务必考虑文章的整体文脉和背景**。

    **评分标准**：
    - 强烈负面 (-5): 极端消极
    - 负面 (-3 ~ -4): 较为消极
    - 轻微负面 (-1 ~ -2): 略微消极
    - 中立 (0): 纯粹的事实陈述，不带有任何情感倾向。
    - 轻微正面 (+1 ~ +2): 略微积极
    - 正面 (+3 ~ +4): 较为积极
    - 强烈正面 (+5): 极端积极

    **请严格分析以下新闻文章的所有句子，并为每个句子提供一个情感分数。所有提供的句子都必须进行情感分析，不得遗漏任何句子**。
    **新闻文章的标题:** {title}
    **新闻文章的句子列表:**
    {sentences}
    **在情感难以明确分类的情况下，请提供您对情感评分的最佳估计，并仅以数字形式返回。**
    """
)

In [17]:
# 체인 구성
sentiment_chain = LLMChain(prompt=prompt_template, llm=llm)

C:\Users\82104\AppData\Local\Temp\ipykernel_23936\1462471065.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  sentiment_chain = LLMChain(prompt=prompt_template, llm=llm)


## 데이터 로드

In [18]:
import json

# JSON 파일 로드
file_path = "../data/smaple1.json"
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

print(f"총 {len(data)}개의 기사 로드 완료")


총 3개의 기사 로드 완료


## 감성분석 수행 함수

In [19]:
def analyze_sentiment(article):
    """기사 전체를 입력하고 문장별 감성 점수를 분석하는 함수"""
    time.sleep(1.5)  # Rate Limit 방지를 위해 대기
    try:
        # 감성 분석을 위한 문장들과 제목을 prompt에 전달
        response = sentiment_chain.run({"sentences": "\n".join(article["sentences"]), "title": article["title"]})
        # 감성 분석 결과 반환
        return response.strip()  # 응답 내용 반환
        # 요청 값 로그 출력
        print(f"API 요청 내용: \n{sentiment_chain.prompt.format(sentences=article['sentences'])}")

        # 실제 응답 내용
        print(f"API 응답 내용: {response}")

        # 응답 내용 sentiment_responses에 저장
        article["sentiment_responses"] = response.content.strip()

        # 결과 반환 (지피티 응답은 항상 저장)
        return article
    except Exception as e:
        article["sentiment_responses"] = f"Error: {str(e)}"  # 오류 메시지 저장
        return article  # 오류 발생 시에도 응답 내용 저장

def process_article(article):
    """GPT 응답 후처리"""
    sentiment_response = article.get("sentiment_responses", "")

    if not sentiment_response:
        return None  # 응답이 없으면 건너뜁니다.

    article["sentiment_responses"] = sentiment_response

    sentiment_scores = []
    for line in sentiment_response.split("\n"):
        try:
            score = float(line.strip())  # 감성 점수만 추출
            sentiment_scores.append(score)
        except (ValueError, IndexError):
            continue  # 오류가 발생하면 해당 문장 건너뜁니다.

    # 평균 감성 점수 계산
    article["sentiment_scores"] = sentiment_scores
    article["avg_sentiment_score"] = round(sum(sentiment_scores) / len(sentiment_scores), 2) if sentiment_scores else None

    return article

def run_sentiment_analysis(data, output_file):
    """JSON 파일을 로드하여 감성 분석을 병렬 실행하고 결과를 저장하는 함수"""
    results = []

    # 기사별 병렬 실행 (최대 3개 스레드 사용)
    with ThreadPoolExecutor(max_workers=3) as executor:
        future_to_article = {executor.submit(analyze_sentiment, article): article for article in data}

        for future in as_completed(future_to_article):
            article = future_to_article[future]  # 원본 기사 가져오기
            processed_article = process_article(article)  # 후처리
            if processed_article:  # 후처리된 결과가 있으면 추가
                results.append(processed_article)

    # 모든 결과를 파일로 저장
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=4)
    print(f" 전체 기사 감성 분석 완료. {output_file}")

In [20]:
output_path = "../data/sentiment_analysis_results-1.json"
run_sentiment_analysis(data, output_path)

 전체 기사 감성 분석 완료. ../data/sentiment_analysis_results-1.json
